In [21]:
import torch
from torch import nn
from d2l import torch as d2l

## 二维互相关计算

In [22]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros([X.shape[0]-h+1, X.shape[1]-w+1])
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]], dtype=torch.float32)
K = torch.tensor([[0, 1], [2, 3]], dtype=torch.float32)

Y = corr2d(X, K)
Y


tensor([[19., 25.],
        [37., 43.]])

## 卷积层

In [23]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return corr2d(X, self.weight) + self.bias


## 图像边缘监测

In [24]:
# 图像生成
X = torch.ones([6, 8])
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [25]:
# 卷积核构造
K = torch.tensor([[1., -1.]]) # 只能监测垂直边缘
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [26]:
corr2d(X.T, K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 学习卷积核

In [28]:
# 构造二维卷积层
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape([1, 1, 6, 8])
Y = Y.reshape([1, 1, 6, 7])
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr*conv2d.weight.grad

    if (i+1)%2 == 0:
        print(f'epoch: {i+1}, loss: {l.sum():.3f}')

epoch: 2, loss: 13.019
epoch: 4, loss: 2.950
epoch: 6, loss: 0.808
epoch: 8, loss: 0.264
epoch: 10, loss: 0.097


In [30]:
conv2d.weight.data.reshape([1,2])

tensor([[ 1.0145, -0.9529]])